In [31]:
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw

mesh = ngs.Mesh('team10_2d.vol')
mesh.Curve(2)
# mesh.Refine()

print("using 3d mesh with ne=", mesh.ne, " elements and nv=", mesh.nv, " points")

p = 2

using 3d mesh with ne= 0  elements and nv= 3846  points


In [36]:
# Draw(mesh)

In [64]:
p = 1

rot = ngs.CF((0,1,-1,0), dims=(2,2))
def curl2d(a):
    return rot*ngs.grad(a)

H1 = ngs.H1(mesh, order = p)
u,v = H1.TnT()

K = ngs.BilinearForm(H1)
K += ngs.curl(u)*ngs.curl(v)*ngs.dx()


total_current = 1000/0.0025
cfJ ngs.= mesh.MaterialCF({"coil_plus": total_current, "coil_minus": -total_current}, default = 0)
# cfJ = ngs.CF({"coil_plus": total_current, "coil_minus": -total_current})

f = ngs.LinearForm(H1)
f += cfJ*ngs.curl(v)*ngs.dxngs.()

NgException: SymbolicLFI needs scalar-valued CoefficientFunction

: 

In [49]:
total_current

400000.0

In [33]:
from ngsolve.krylovspace import CGSolver



# Weak form
T, Tstar = HcurlCoil.TnT()
Kt = ngs.BilinearForm(ngs.curl(Tstar)*ngs.curl(T)*ngs.dx)
    
# Assembly
with ngs.TaskManager(): Kt.Assemble()

r = - Kt.mat * gfuDirichlet.vec

# Solving
maxres = 1e-5
maxit = 1000
                              
Tsol = ngs.GridFunction(HcurlCoil)
print("Solving...")

with ngs.TaskManager():
    iterativeSolver = CGSolver(Kt.mat, freedofs = HcurlCoil.FreeDofs(), atol  = maxres,  maxiter = maxit)
    Tsol.vec.data = iterativeSolver * r + gfuDirichlet.vec

# if len(iterativeSolver.residuals) == maxit: print("... Failure!")
# else: print("... Success!")
print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")

# Draw(ngs.curl(Tsol), mesh, vectors = { "grid_size" : 150},clipping = {"x" : 0, "y" : 0, "z" : -1, "dist" : 0})
with ngs.TaskManager():
    flux_bd = ngs.Integrate(ngs.curl(Tsol) * n, mesh, definedon = mesh.Boundaries("coil_cut_2"))
print(flux_bd)

Solving...
Number of iterations = 357/1000 | Residual = 9.493271747018348e-06
0.0


In [34]:
# Hcurl = ngs.HCurl(mesh, order = p, nograds = True)
Hcurl = ngs.HCurl(mesh, order = p, nograds = True, dirichlet = 'ambient_face')
u,v = Hcurl.TnT()

K = ngs.BilinearForm(ngs.curl(u)*ngs.curl(v)*ngs.dx)
c = ngs.Preconditioner(K, type = "local")

with ngs.TaskManager(): K.Assemble()

f = ngs.LinearForm(Hcurl)
f +=  ngs.curl(Tsol)* ngs.curl(v) * ngs.dx

with ngs.TaskManager(): f.Assemble()

Hs = ngs.GridFunction(Hcurl)
print("Solving...")

with ngs.TaskManager():
    iterativeSolver = CGSolver(K.mat, c.mat, atol  = maxres,  maxiter = maxit)
    # iterativeSolver = CGSolver(K.mat, freedofs = Hcurl.FreeDofs(), tol  = maxres,  maxiter = maxit)
    Hs.vec.data = iterativeSolver * f.vec

# if len(iterativeSolver.residuals) == maxit: print("... Failure!")
# else: print("... Success!")
print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")

# Draw(ngs.curl(Hs), mesh, vectors = { "grid_size" : 150},clipping = {"x" : 0, "y" : 0, "z" : -1, "dist" : 0})

with ngs.TaskManager():
    flux_bd = ngs.Integrate(ngs.curl(Hs) * n, mesh, definedon = mesh.Boundaries("coil_cut_2"))
print(flux_bd)

Solving...
Number of iterations = 1000/1000 | Residual = nan
0.0


In [35]:
# Draw(ngs.curl(Hs), mesh, vectors = { "grid_size" : 500},clipping = {"x" : 0, "y" : 0, "z" : -1, "dist" : 0})